In [1]:
import numpy as np
import pandas as pd
import warnings 
warnings.filterwarnings('ignore')
from sklearn.preprocessing import Imputer  
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant


Train Dataset

In [2]:
train = pd.read_csv('Task2_Train_Data_2.csv', low_memory = False, delimiter =',')
train.head()

,Buy_sesID,Cli_ItemID,ttl_Cli,AVG_CLI,Item_CLI,Fre_Cli_Itm,IN_Fre_Ctg,Cli_Wd,Cli_T,Dur_Sec,Cli_buy,Item_bought_P90,Fir_Cli,Las_Cli,Item_avg_pric,AVG_Pric,Pri_ctg_P90,Pred_Target
0,10000004,214851326,8,2.0,3,1,1,Friday,3,255,3,1,0,0,1046.0,1923.025449,4920.6,1
1,10000004,214853248,8,2.0,1,0,0,Friday,3,0,1,1,0,0,837.0,2061.555292,48380.5,0
2,10000004,214853094,8,2.0,1,0,1,Friday,3,0,1,1,0,0,4188.0,1923.025449,4920.6,0
3,10000004,214853090,8,2.0,3,1,1,Friday,3,3117,3,1,1,1,4188.0,1923.025449,4920.6,1
4,10000024,214567333,4,1.0,1,0,1,Sunday,2,0,1,1,0,0,837.0,2061.555292,48380.5,1


convert categorical string to numeric

In [3]:
train['Cli_Wd'] = train['Cli_Wd'].map (lambda x:1 if ('Mon' in x)
                                            else 2 if ('Tues' in x)
                                            else 3 if ('Wed' in x)
                                            else 4 if ('Thur' in x)
                                            else 5 if ('Fri' in x)
                                            else 6 if ('Sat' in x)
                                            else 7  )



Missing Data Imputation

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355403 entries, 0 to 355402
Data columns (total 18 columns):
Buy_sesID          355403 non-null int64
Cli_ItemID         355403 non-null int64
ttl_Cli            355403 non-null int64
AVG_CLI            355403 non-null float64
Item_CLI           355403 non-null int64
Fre_Cli_Itm        355403 non-null int64
IN_Fre_Ctg         355403 non-null int64
Cli_Wd             355403 non-null int64
Cli_T              355403 non-null int64
Dur_Sec            355403 non-null int64
Cli_buy            355403 non-null int64
Item_bought_P90    355403 non-null int64
Fir_Cli            355403 non-null int64
Las_Cli            355403 non-null int64
Item_avg_pric      350387 non-null float64
AVG_Pric           355402 non-null float64
Pri_ctg_P90        355402 non-null float64
Pred_Target        355403 non-null int64
dtypes: float64(4), int64(14)
memory usage: 48.8 MB


In [5]:
imputer = Imputer(strategy = "median")

imputer.fit(train)
X = imputer.transform(train)
train_tr = pd.DataFrame(X, columns = train.columns)


In [6]:
train_tr['below_ctg_avg_pric'] = train_tr.apply(lambda x:1 if x.Item_avg_pric <= x.AVG_Pric
                                                else 0, axis = 1).astype(np.float64)

train_tr['below_ctg_P90'] = train_tr.apply(lambda x:1 if x.Item_avg_pric <= x.Pri_ctg_P90
                                          else 0, axis = 1).astype(np.float64)
train_tr = train_tr.convert_objects(convert_numeric=True)

In [7]:
train_tr.describe()

,Buy_sesID,Cli_ItemID,ttl_Cli,AVG_CLI,Item_CLI,Fre_Cli_Itm,IN_Fre_Ctg,Cli_Wd,Cli_T,Dur_Sec,Cli_buy,Item_bought_P90,Fir_Cli,Las_Cli,Item_avg_pric,AVG_Pric,Pri_ctg_P90,Pred_Target,below_ctg_avg_pric,below_ctg_P90
count,3.554030e+05,3.554030e+05,355403.000000,355403.000000,355403.000000,355403.000000,355403.000000,355403.000000,355403.000000,355403.000000,355403.000000,355403.000000,355403.000000,355403.000000,355403.000000,355403.000000,355403.000000,355403.000000,355403.000000,355403.000000
mean,1.045504e+07,2.147903e+08,13.992161,1.363093,1.371854,0.184478,0.794318,4.392650,2.841059,190.627941,1.458229,0.756828,0.344792,0.344792,2162.095967,1952.609708,45945.476872,0.459208,0.779850,0.996286
std,5.034714e+05,1.024447e+05,14.644250,0.530975,0.866472,0.387874,0.404200,2.391107,0.859991,691.758609,0.830255,0.428999,0.475301,0.475301,5676.458780,371.880315,17822.428264,0.498334,0.414348,0.060830
min,9.195066e+06,2.145073e+08,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,26.080124,821.100000,0.000000,0.000000,0.000000
25%,1.007078e+07,2.147161e+08,5.000000,1.000000,1.000000,0.000000,1.000000,1.000000,2.000000,0.000000,1.000000,1.000000,0.000000,0.000000,523.000000,1923.025449,48380.500000,0.000000,1.000000,1.000000
50%,1.049526e+07,2.148512e+08,10.000000,1.200000,1.000000,0.000000,1.000000,5.000000,3.000000,0.000000,1.000000,1.000000,0.000000,0.000000,941.000000,2061.555292,48380.500000,0.000000,1.000000,1.000000
75%,1.088816e+07,2.148542e+08,17.000000,1.500000,1.000000,0.000000,1.000000,7.000000,4.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1570.000000,2061.555292,48380.500000,1.000000,1.000000,1.000000
max,1.156001e+07,2.149884e+08,197.000000,16.000000,28.000000,1.000000,1.000000,7.000000,4.000000,17780.000000,3.000000,1.000000,1.000000,1.000000,293111.000000,5108.916129,106709.400000,1.000000,1.000000,1.000000


In [8]:
train_tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355403 entries, 0 to 355402
Data columns (total 20 columns):
Buy_sesID             355403 non-null float64
Cli_ItemID            355403 non-null float64
ttl_Cli               355403 non-null float64
AVG_CLI               355403 non-null float64
Item_CLI              355403 non-null float64
Fre_Cli_Itm           355403 non-null float64
IN_Fre_Ctg            355403 non-null float64
Cli_Wd                355403 non-null float64
Cli_T                 355403 non-null float64
Dur_Sec               355403 non-null float64
Cli_buy               355403 non-null float64
Item_bought_P90       355403 non-null float64
Fir_Cli               355403 non-null float64
Las_Cli               355403 non-null float64
Item_avg_pric         355403 non-null float64
AVG_Pric              355403 non-null float64
Pri_ctg_P90           355403 non-null float64
Pred_Target           355403 non-null float64
below_ctg_avg_pric    355403 non-null float64
below_ctg_P90  

In [9]:
corr_matrix = train_tr.corr()
corr_matrix["Pred_Target"].sort_values(ascending=False)

Pred_Target           1.000000
Las_Cli               0.291775
Fir_Cli               0.291775
Cli_buy               0.266504
Item_CLI              0.232301
Fre_Cli_Itm           0.218491
Item_bought_P90       0.208933
Dur_Sec               0.163639
IN_Fre_Ctg            0.163089
Cli_ItemID            0.142385
AVG_CLI               0.067386
AVG_Pric              0.057778
Pri_ctg_P90           0.042108
Buy_sesID             0.016876
Cli_Wd                0.002600
below_ctg_P90         0.002521
below_ctg_avg_pric    0.001613
Item_avg_pric        -0.015320
Cli_T                -0.043019
ttl_Cli              -0.226007
Name: Pred_Target, dtype: float64

In [10]:
X = add_constant(train_tr)
pd.Series([variance_inflation_factor(X.values, i) 
               for i in range(X.shape[1])], 
              index=X.columns)


const                 0.000000
Buy_sesID             1.059847
Cli_ItemID            1.278985
ttl_Cli               1.233813
AVG_CLI               2.237990
Item_CLI              3.865485
Fre_Cli_Itm           5.655106
IN_Fre_Ctg            1.120053
Cli_Wd                1.058873
Cli_T                 1.011149
Dur_Sec               1.508262
Cli_buy               6.491980
Item_bought_P90       1.437310
Fir_Cli                    inf
Las_Cli                    inf
Item_avg_pric         1.952806
AVG_Pric              1.204907
Pri_ctg_P90           1.135789
Pred_Target           1.253733
below_ctg_avg_pric    1.438044
below_ctg_P90         1.520745
dtype: float64

In [11]:
train_tr.to_csv('Task2_Train_Data_pro.csv', sep = ',')

Validation dataset

In [12]:
Valid = pd.read_csv('Task2_Validation_Data_2.csv', low_memory = False, delimiter =',')
Valid.head()

,Buy_sesID,Cli_ItemID,ttl_Cli,AVG_CLI,Item_CLI,Fre_Cli_Itm,IN_Fre_Ctg,Cli_Wd,Cli_T,Dur_Sec,Cli_buy,Item_bought_P90,Fir_Cli,Las_Cli,Item_avg_pric,AVG_Pric,Pri_ctg_P90,Pred_Target
0,11255564,214854358,5,2.5,2,0,1,Thursday,3,149,3,1,1,1,1360.0,1923.025449,4920.6,0
1,11255564,214859126,5,2.5,3,1,1,Thursday,3,946,3,1,1,1,1570.0,1923.025449,4920.6,1
2,11255594,214712244,3,1.0,1,0,1,Wednesday,4,0,1,1,1,1,947.0,2061.555292,48380.5,1
3,11255594,214676480,3,1.0,1,0,1,Wednesday,4,0,1,1,1,1,5538.0,2061.555292,48380.5,0
4,11255594,214690839,3,1.0,1,0,1,Wednesday,4,0,1,0,0,0,686.0,2061.555292,48380.5,0


In [13]:
Valid['Cli_Wd'] = Valid['Cli_Wd'].map (lambda x:1 if ('Mon' in x)
                                            else 2 if ('Tues' in x)
                                            else 3 if ('Wed' in x)
                                            else 4 if ('Thur' in x)
                                            else 5 if ('Fri' in x)
                                            else 6 if ('Sat' in x)
                                            else 7  )

Item_pric_md_V = Valid['Item_avg_pric'].median()
Valid['Item_avg_pric'].fillna(Item_pric_md_V)

Valid.dropna()
Valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55704 entries, 0 to 55703
Data columns (total 18 columns):
Buy_sesID          55704 non-null int64
Cli_ItemID         55704 non-null int64
ttl_Cli            55704 non-null int64
AVG_CLI            55704 non-null float64
Item_CLI           55704 non-null int64
Fre_Cli_Itm        55704 non-null int64
IN_Fre_Ctg         55704 non-null int64
Cli_Wd             55704 non-null int64
Cli_T              55704 non-null int64
Dur_Sec            55704 non-null int64
Cli_buy            55704 non-null int64
Item_bought_P90    55704 non-null int64
Fir_Cli            55704 non-null int64
Las_Cli            55704 non-null int64
Item_avg_pric      54545 non-null float64
AVG_Pric           55704 non-null float64
Pri_ctg_P90        55704 non-null float64
Pred_Target        55704 non-null int64
dtypes: float64(4), int64(14)
memory usage: 7.6 MB


In [14]:
imputer = Imputer(strategy = "median")

imputer.fit(Valid)
X = imputer.transform(Valid)
Valid_tr = pd.DataFrame(X, columns = Valid.columns)
Valid_tr['below_ctg_avg_pric'] = Valid_tr.apply(lambda x:1 if x.Item_avg_pric <= x.AVG_Pric
                                                else 0, axis = 1).astype(np.float64)

Valid_tr['below_ctg_P90'] = Valid_tr.apply(lambda x:1 if x.Item_avg_pric <= x.Pri_ctg_P90
                                          else 0, axis = 1).astype(np.float64)

In [15]:
Valid_tr.describe()

,Buy_sesID,Cli_ItemID,ttl_Cli,AVG_CLI,Item_CLI,Fre_Cli_Itm,IN_Fre_Ctg,Cli_Wd,Cli_T,Dur_Sec,Cli_buy,Item_bought_P90,Fir_Cli,Las_Cli,Item_avg_pric,AVG_Pric,Pri_ctg_P90,Pred_Target,below_ctg_avg_pric,below_ctg_P90
count,5.570400e+04,5.570400e+04,55704.000000,55704.000000,55704.000000,55704.000000,55704.000000,55704.000000,55704.000000,55704.000000,55704.000000,55704.000000,55704.000000,55704.000000,55704.000000,55704.000000,55704.000000,55704.000000,55704.000000,55704.000000
mean,1.141138e+07,2.147884e+08,13.888841,1.350477,1.359471,0.181549,0.753213,4.250844,2.856563,194.383850,1.457364,0.620458,0.362128,0.362128,2685.196593,1924.906627,43920.950934,0.463863,0.743322,0.995584
std,8.742853e+04,1.046440e+05,14.789570,0.494561,0.821326,0.385476,0.431145,1.313135,0.875250,688.338454,0.832537,0.485277,0.480620,0.480620,6183.850269,417.458490,20715.860130,0.498697,0.436804,0.066308
min,1.125556e+07,2.145073e+08,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,312.094944,2760.400000,0.000000,0.000000,0.000000
25%,1.133426e+07,2.147122e+08,5.000000,1.000000,1.000000,0.000000,1.000000,3.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,523.000000,1923.025449,41783.000000,0.000000,0.000000,1.000000
50%,1.141495e+07,2.148513e+08,9.000000,1.200000,1.000000,0.000000,1.000000,4.000000,3.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1046.000000,2061.555292,48380.500000,0.000000,1.000000,1.000000
75%,1.148886e+07,2.148548e+08,17.000000,1.500000,1.000000,0.000000,1.000000,5.000000,4.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1806.000000,2061.555292,48380.500000,1.000000,1.000000,1.000000
max,1.156212e+07,2.149811e+08,146.000000,12.000000,18.000000,1.000000,1.000000,7.000000,4.000000,11394.000000,3.000000,1.000000,1.000000,1.000000,204099.000000,3059.982833,106709.400000,1.000000,1.000000,1.000000


In [16]:
Valid_tr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55704 entries, 0 to 55703
Data columns (total 20 columns):
Buy_sesID             55704 non-null float64
Cli_ItemID            55704 non-null float64
ttl_Cli               55704 non-null float64
AVG_CLI               55704 non-null float64
Item_CLI              55704 non-null float64
Fre_Cli_Itm           55704 non-null float64
IN_Fre_Ctg            55704 non-null float64
Cli_Wd                55704 non-null float64
Cli_T                 55704 non-null float64
Dur_Sec               55704 non-null float64
Cli_buy               55704 non-null float64
Item_bought_P90       55704 non-null float64
Fir_Cli               55704 non-null float64
Las_Cli               55704 non-null float64
Item_avg_pric         55704 non-null float64
AVG_Pric              55704 non-null float64
Pri_ctg_P90           55704 non-null float64
Pred_Target           55704 non-null float64
below_ctg_avg_pric    55704 non-null float64
below_ctg_P90         55704 non-null

In [17]:
Valid_tr.to_csv('Task2_Validation_Data_pro.csv', sep = ',')